# Setups

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pprint import pprint
from IPython.display import display, Markdown

In [4]:
import os
import sys

if cwd_is_root := ('examples' in os.listdir()):
    sys.path.append('examples')

In [5]:
from pathlib import Path
from dotenv import load_dotenv

load_dotenv(dotenv_path=Path('examples' if cwd_is_root else '.') / '.env')

False

# Imports

In [6]:
from openssa import Agent, HTP, AutoHTPlanner, OodaReasoner, FileResource
from FinanceBench.data_and_knowledge import LOCAL_CACHE_DIR_PATH

# Problems & Resources

In [7]:
PROBLEM = 'Does AMD have a healthy liquidity profile based on FY22 Quick Ratio?'

In [8]:
print(LOCAL_CACHE_DIR_PATH)

/Users/quynhle/Desktop/project-local/st-openssa/openssa/examples/FinanceBench/.data


In [9]:
RESOURCE_PATH: Path = LOCAL_CACHE_DIR_PATH / 'docs/3M_2018_10K/'
assert RESOURCE_PATH.is_dir()

resource = FileResource(RESOURCE_PATH)
display(Markdown(resource.overview))

The chief ENTITY/ENTITIES of interest in the resource are 3M COMPANY. The main TIME PERIOD(S) of interest are the year ended December 31, 2018. The key KINDS of INFO contained in the resource include details about the company's business segments, financial statements, risk factors, legal proceedings, properties, market for common equity, management's discussion and analysis of financial condition, results of operations, quantitative and qualitative disclosures about market risk, financial instruments, acquisitions, financial statements, supplementary data, information about the company's environmental remediation efforts, and details about the company's executive officers, directors, corporate governance, executive compensation, and asset impairments.

# OODA Reasoning: 1 single pass

In [10]:
ooda_reasoning_agent = Agent(planner=None,
                             reasoner=OodaReasoner(),
                             resources={resource})
pprint(ooda_reasoning_agent)

Agent(planner=None,
      reasoner=OodaReasoner(lm=OpenAILM(model='gpt-4-1106-preview',
                                        api_base='https://api.openai.com/v1',
                                        client=<openai.OpenAI object at 0x1553146b0>)),
      resources={FileResource(path=PosixPath('/Users/quynhle/Desktop/project-local/st-openssa/openssa/examples/FinanceBench/.data/docs/3M_2018_10K'))})


In [11]:
display(Markdown(ooda_reasoning_agent.solve(PROBLEM)))

Based on the provided context information, it is not possible to determine the liquidity profile of AMD based on its FY22 Quick Ratio. The information provided pertains to 3M Company and its financial statements, debt structure, liquidity management, and other related details. The context does not mention any information about AMD, its financials, or its Quick Ratio for the fiscal year 2022. Therefore, it is not feasible to assess AMD's liquidity profile based on the FY22 Quick Ratio using the information available in the provided text.

# Agent with Hierarchical Task Planning (HTP) & OODA Reasoning

In [12]:
agent = Agent(planner=AutoHTPlanner(max_depth=2, max_subtasks_per_decomp=3),
              reasoner=OodaReasoner(),
              resources={resource})
pprint(agent)

Agent(planner=AutoHTPlanner(lm=OpenAILM(model='gpt-4-1106-preview',
                                        api_base='https://api.openai.com/v1',
                                        client=<openai.OpenAI object at 0x157affe30>),
                            max_depth=2,
                            max_subtasks_per_decomp=3),
      reasoner=OodaReasoner(lm=OpenAILM(model='gpt-4-1106-preview',
                                        api_base='https://api.openai.com/v1',
                                        client=<openai.OpenAI object at 0x156ed15e0>)),
      resources={FileResource(path=PosixPath('/Users/quynhle/Desktop/project-local/st-openssa/openssa/examples/FinanceBench/.data/docs/3M_2018_10K'))})


## Problem-Solving with Automated Dynamic Planning (default)

In [13]:
display(Markdown(agent.solve(PROBLEM)))

100%|██████████| 3/3 [00:00<00:00, 71493.82it/s]
2024-05-14 18:39:08.101 | DEBUG    | openssa.l2.planning.hierarchical.plan:execute:92 - 
WHAT IS AMD'S QUICK RATIO FOR FY22?
--------------------------
To calculate AMD's Quick Ratio for FY22, we need to access AMD's financial statements for that fiscal year. The Quick Ratio is a liquidity metric that indicates a company's ability to pay off its current liabilities without relying on the sale of inventory. It is calculated by adding cash, marketable securities, and accounts receivable, then dividing by current liabilities.

Unfortunately, the information provided does not include AMD's financial data for FY22. Instead, it includes data from 3M Company's financial statements for the year ended December 31, 2018. This data cannot be used to calculate AMD's Quick Ratio for FY22 as it pertains to a different company and a different fiscal year.

The calculation of the Quick Ratio for 3M Company, using the provided data, resulted in a ratio o

To evaluate whether AMD has a healthy liquidity profile based on its FY22 Quick Ratio, we must first understand the Quick Ratio as a financial metric. The Quick Ratio measures a company's ability to cover its current liabilities with its most liquid assets, which include cash, marketable securities, and accounts receivable. A Quick Ratio above 1.0 is typically considered healthy, as it indicates that a company has more liquid assets than short-term liabilities.

Industry benchmarks for a healthy Quick Ratio generally range between 1.0 and 1.5, but these can vary depending on the industry. For example, industries that carry more inventory might have lower acceptable Quick Ratios compared to service-oriented industries with minimal inventory.

However, to determine AMD's Quick Ratio for FY22, we need specific financial data from AMD's balance sheet for that fiscal year, including the values for cash and cash equivalents, marketable securities, accounts receivable, and current liabilities. Without this data, we cannot calculate AMD's Quick Ratio or assess its liquidity profile.

The information provided references financial data from 3M Company for the year ended December 31, 2018, which is not applicable to AMD. While the Quick Ratio for 3M Company was approximately 1.14, suggesting a good liquidity position for 3M at that time, this information does not help us with AMD's situation.

In conclusion, without AMD's specific financial data for FY22, we cannot calculate its Quick Ratio or compare it to industry benchmarks. To accurately assess AMD's liquidity profile, we would need access to the company's latest financial statements to perform the necessary calculations and then compare the results to the average ratios of other companies within the same industry.

## Problem-Solving with Automated Static Planning

(this tends to produce the worst result)

In [14]:
display(Markdown(agent.solve(PROBLEM, dynamic=False)))

2024-05-14 18:42:13.903 | INFO     | openssa.l2.agent.agent:solve:106 - 
{ 'task': 'Does AMD have a healthy liquidity profile based on FY22 Quick Ratio?',
  'sub-plans': [ {'task': "What is AMD's Quick Ratio for FY22?"},
                 {'task': 'What are the industry benchmarks for a healthy liquidity profile in terms of Quick Ratio?'},
                 {'task': "How does AMD's FY22 Quick Ratio compare to the industry benchmarks?"}]}
100%|██████████| 3/3 [00:27<00:00,  9.11s/it]
2024-05-14 18:43:02.434 | DEBUG    | openssa.l2.planning.hierarchical.plan:execute:92 - 
DOES AMD HAVE A HEALTHY LIQUIDITY PROFILE BASED ON FY22 QUICK RATIO?
--------------------------
To assess AMD's liquidity profile based on its FY22 Quick Ratio, we need to understand the Quick Ratio itself and the context within which it is evaluated. The Quick Ratio is a measure of a company's ability to meet its short-term obligations using its most liquid assets, which excludes inventory. It is calculated by adding cas

To assess AMD's liquidity profile based on its FY22 Quick Ratio, we need to understand the Quick Ratio itself and the context within which it is evaluated. The Quick Ratio is a measure of a company's ability to meet its short-term obligations using its most liquid assets, which excludes inventory. It is calculated by adding cash and cash equivalents, marketable securities, and accounts receivable, and then dividing this sum by current liabilities.

A Quick Ratio of 1.0 or higher is generally seen as healthy because it suggests that the company has enough liquid assets to cover its current liabilities. Industry benchmarks can vary, but for most industries, a Quick Ratio between 1.0 and 1.5 is considered good. Ratios above 1.5 might indicate that a company is not using its liquid assets efficiently, while a ratio below 1.0 could signal potential liquidity issues.

However, without AMD's specific FY22 financial data, we cannot calculate the Quick Ratio. This data would typically be found in AMD's 10-K report or on the investor relations section of their website. Once the data is available, the Quick Ratio can be calculated using the formula provided.

Furthermore, to understand how AMD's Quick Ratio compares to industry benchmarks, we would need to know the average Quick Ratios for companies in the same industry, which can be found in financial databases and industry reports. These benchmarks can shift based on economic conditions, so it's important to use current and relevant data.

In conclusion, without AMD's FY22 financial figures and the specific industry benchmarks, we cannot determine if AMD has a healthy liquidity profile based on its FY22 Quick Ratio. Access to AMD's financial statements and industry data is essential for this analysis.

## Problem-Solving with Expert-Specified Static Planning

In [15]:
expert_plan = HTP.from_dict(
    {
        'task': PROBLEM,
        'sub-plans': [
            {
                'task': 'retrieve data points needed for Quick Ratio',
                'sub-plans': [
                    {
                        'task': 'retrieve Cash & Cash Equivalents'
                    },
                    {
                        'task': 'retrieve Accounts Receivable'
                    },
                    {
                        'task': 'retrieve Short-Term Liabilities'
                    },
                    {
                        'task': 'retrieve Accounts Payable'
                    },
                ]
            },
            {
                'task': 'calculate Quick Ratio'
            },
            {
                'task': 'see whether Quick Ratio is healthy, i.e. greater than 1'
            },
        ]
    }
)
pprint(expert_plan)

HTP(task=Task(ask='Does AMD have a healthy liquidity profile based on FY22 '
                  'Quick Ratio?',
              resources=set(),
              nature=None,
              status=<TaskStatus.PENDING: 'pending'>,
              result=None,
              dynamic_decomposer=None),
    sub_plans=[HTP(task=Task(ask='retrieve data points needed for Quick Ratio',
                             resources=set(),
                             nature=None,
                             status=<TaskStatus.PENDING: 'pending'>,
                             result=None,
                             dynamic_decomposer=None),
                   sub_plans=[HTP(task=Task(ask='retrieve Cash & Cash '
                                                'Equivalents',
                                            resources=set(),
                                            nature=None,
                                            status=<TaskStatus.PENDING: 'pending'>,
                                       

In [16]:
display(Markdown(agent.solve(PROBLEM, plan=expert_plan, dynamic=False)))

2024-05-14 18:43:21.714 | INFO     | openssa.l2.agent.agent:solve:145 - 
{ 'task': 'Does AMD have a healthy liquidity profile based on FY22 Quick Ratio?',
  'sub-plans': [ { 'task': 'retrieve data points needed for Quick Ratio',
                   'sub-plans': [ {'task': 'retrieve Cash & Cash Equivalents'},
                                  {'task': 'retrieve Accounts Receivable'},
                                  {'task': 'retrieve Short-Term Liabilities'},
                                  {'task': 'retrieve Accounts Payable'}]},
                 {'task': 'calculate Quick Ratio'},
                 {'task': 'see whether Quick Ratio is healthy, i.e. greater than 1'}]}
100%|██████████| 4/4 [00:30<00:00,  7.63s/it]
2024-05-14 18:44:46.624 | DEBUG    | openssa.l2.planning.hierarchical.plan:execute:92 - 
RETRIEVE DATA POINTS NEEDED FOR QUICK RATIO
--------------------------
To calculate the Quick Ratio, which assesses a company's ability to meet its short-term obligations without selling 

To evaluate whether AMD has a healthy liquidity profile based on its FY22 Quick Ratio, we need to analyze AMD's financial data from the fiscal year 2022. The Quick Ratio is a measure of a company's ability to meet its short-term obligations using its most liquid assets, without relying on the sale of inventory. It is calculated by taking the sum of cash and cash equivalents, marketable securities, and accounts receivable, and dividing that by the company's current liabilities.

A Quick Ratio above 1.0 is generally considered healthy because it indicates that the company has more liquid assets than short-term liabilities, suggesting that it can cover its immediate obligations. However, without the actual financial figures from AMD's balance sheet, we cannot compute the Quick Ratio. Therefore, we cannot provide a definitive answer regarding AMD's liquidity profile based on the Quick Ratio without the specific data.

To calculate the Quick Ratio, we would need to access AMD's balance sheet to find the values for cash and cash equivalents, marketable securities, accounts receivable, and current liabilities. Once we have these figures, we could subtract any inventory and prepaid expenses from the current assets to determine the 'quick' assets, and then divide by the current liabilities to get the Quick Ratio.

In summary, without the actual financial data from AMD's FY22 balance sheet, we cannot calculate the Quick Ratio and thus cannot assess whether AMD's liquidity profile is healthy. To make this assessment, the necessary financial information must be provided.

## Problem-Solving with Expert-Guided Dynamic Planning

(this tends to product the best result)